# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

### SOLUTION PROCESS

Get the POSTATE of CITING and CITED columns for citations data  

In [8]:
postate_results = citations.alias("c").join(patents.alias("p1"), col("c.CITED") == col("p1.PATENT"), "inner") \
    .join(patents.alias("p2"), col("c.CITING") == col("p2.PATENT"), "inner") \
    .select(col("c.CITING"), col("p2.POSTATE"),col("c.CITED"), col("p1.POSTATE"))
postate_results.show(5)

+-------+-------+-------+-------+
| CITING|POSTATE|  CITED|POSTATE|
+-------+-------+-------+-------+
|3858242|     MI|3319261|     OH|
|3858242|     MI|3707004|     WI|
|3858242|     MI|3668705|     WI|
|3858243|   null|3574238|     WI|
|3858243|   null|3146465|     MI|
+-------+-------+-------+-------+
only showing top 5 rows



Get results where the POSTATE is same and not null

In [9]:
state_same = citations.alias("c").join(patents.alias("p1"), col("c.CITED") == col("p1.PATENT"), "inner") \
    .join(patents.alias("p2"), col("c.CITING") == col("p2.PATENT"), "inner") \
    .filter((col("p1.POSTATE").isNotNull()) & (col("p1.POSTATE") != '') & (col("p1.POSTATE") == col("p2.POSTATE"))) \
    .select(col("c.CITING"), col("p2.POSTATE"),col("c.CITED"), col("p1.POSTATE"))
state_same.show(5)

+-------+-------+-------+-------+
| CITING|POSTATE|  CITED|POSTATE|
+-------+-------+-------+-------+
|3858241|     MA|3557384|     MA|
|3858247|     CA|3621837|     CA|
|3858247|     CA|3694819|     CA|
|3858262|     MA|3416492|     MA|
|3858262|     MA|3452714|     MA|
+-------+-------+-------+-------+
only showing top 5 rows



Get the total count for individual patents

In [12]:
test = citations.alias("c").join(patents.alias("p1"), col("c.CITED") == col("p1.PATENT"), "inner") \
    .join(patents.alias("p2"), col("c.CITING") == col("p2.PATENT"), "inner") \
    .filter((col("p1.POSTATE").isNotNull()) & (col("p1.POSTATE") != '') & (col("p1.POSTATE") == col("p2.POSTATE"))) \
    .groupBy("c.CITING") \
    .agg(
        count("c.CITED").alias("SAME_STATE")
    )
test.show(5)

+-------+----------+
| CITING|SAME_STATE|
+-------+----------+
|3859627|         1|
|3860191|         1|
|3862577|         1|
|3862844|         1|
|3863090|         1|
+-------+----------+
only showing top 5 rows



Order the results based on count and CITING 

In [13]:
ordered_results = citations.alias("c").join(patents.alias("p1"), col("c.CITED") == col("p1.PATENT"), "inner") \
    .join(patents.alias("p2"), col("c.CITING") == col("p2.PATENT"), "inner") \
    .filter((col("p1.POSTATE").isNotNull()) & (col("p1.POSTATE") != '') & (col("p1.POSTATE") == col("p2.POSTATE"))) \
    .groupBy("c.CITING") \
    .agg(
        count("c.CITED").alias("SAME_STATE")
    ) \
    .orderBy(col("SAME_STATE").desc(), col("c.CITING").asc())
ordered_results.show(10)

+-------+----------+
| CITING|SAME_STATE|
+-------+----------+
|5959466|       125|
|5983822|       103|
|6008204|       100|
|5952345|        98|
|5958954|        96|
|5998655|        96|
|5936426|        94|
|5739256|        90|
|5913855|        90|
|5925042|        90|
+-------+----------+
only showing top 10 rows



#### FINAL RESULT

Join the above results with patent data

In [14]:
result = patents.alias("p").join(ordered_results.alias("r"), col("r.CITING") == col("p.PATENT"), "inner") \
        .select(col("p.*"),col("SAME_STATE")) \
        .orderBy(col("SAME_STATE").desc(), col("p.PATENT").asc())
result.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|       103|
|6008204| 1999|14606|   1998| 